```
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
```

# Modeling PetFinder.my with Logistic Regression

## 1. Load Data 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

import sklearn.linear_model as lm

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.metrics import make_scorer
from sklearn.metrics import cohen_kappa_score # https://www.kaggle.com/aroraaman/quadratic-kappa-metric-explained-in-5-simple-steps

sns.set_style("darkgrid")
sns.set_context("paper")
mpl.style.use("seaborn")

In [ ]:
SOURCE_DATA_FOLDER = "/kaggle/input/petfinder-adoption-prediction/"
TRAIN_IMAGE_FOLDER = SOURCE_DATA_FOLDER + "train_images/"
TRAIN_METADATA_FOLDER = SOURCE_DATA_FOLDER + "train_metadata/"
TRAIN_SENTIMENT_FOLDER = SOURCE_DATA_FOLDER + "train_sentiment/"

BREED_LABELS = SOURCE_DATA_FOLDER + "breed_labels.csv"
COLOR_LABELS = SOURCE_DATA_FOLDER + "color_labels.csv"
STATE_LABELS = SOURCE_DATA_FOLDER + "state_labels.csv"
TRAIN_TABULAR = SOURCE_DATA_FOLDER + "train/train.csv"
TEST_TABULAR = SOURCE_DATA_FOLDER + "test/test.csv"

quadratic_kappa_scorer = make_scorer(cohen_kappa_score, weights='quadratic')

In [ ]:
train_tabular_df = pd.read_csv(TRAIN_TABULAR)
test_tabular_df = pd.read_csv(TEST_TABULAR)

## 2. Preprocess data and transform features

In [ ]:
def getRescuerIDCountMap(data):
    rescuerid_counts = data['RescuerID'].value_counts().to_dict()
    return rescuerid_counts

def encodeRescuerIDCount(data, mapping):
    dataset = data.copy()
    dataset['RescuerID_Count'] = dataset['RescuerID'].map(mapping).fillna(0)
    return dataset

def createRescuerIDCount(data):
    # RescuerID Counts
    dataset = data.copy()
    rescuerid_counts = getRescuerIDCountMap(dataset)
    dataset = encodeRescuerIDCount(dataset, rescuerid_counts)
    return dataset, rescuerid_counts

def createDummyVariables(data):
    dataset = data.copy()
    # fee 0 dummy
    dataset['Flag_Fee_0'] = np.nan
    dataset['Flag_Fee_0'] = dataset['Flag_Fee_0'].mask(dataset['Fee'] == 0, 1)
    dataset['Flag_Fee_0'] = dataset['Flag_Fee_0'].fillna(0)

    # breed dummy variables
    mixed_breed_id = 307

    # pure breeds are where Breed1 == Breed2
    pure_breed_cond = (dataset['Breed1'] != mixed_breed_id) & (dataset['Breed1'] == dataset['Breed2'])
    dataset['Flag_Pure_Breed'] = np.nan
    dataset['Flag_Pure_Breed'] = dataset['Flag_Pure_Breed'].mask(pure_breed_cond, 1)
    dataset['Flag_Pure_Breed'] = dataset['Flag_Pure_Breed'].fillna(0)

    # we define mixed breeds as Breed1 or Breed2 being Mixed-Breed, or Breed1 != Breed2 where Breed1 and Breed2 are not Mixed-Breed
    mixed_breed_cond = ((dataset['Breed1'] == 307) | (dataset['Breed2'] == 307)) | ((dataset['Breed1'] != dataset['Breed2']) & 
                                                                                             (dataset['Breed2'] != 0) & 
                                                                                             (dataset['Breed2'] != 307)& 
                                                                                             (dataset['Breed1'] != 307))
    dataset['Flag_Mixed_Breed'] = np.nan
    dataset['Flag_Mixed_Breed'] = dataset['Flag_Mixed_Breed'].mask(mixed_breed_cond, 1)
    dataset['Flag_Mixed_Breed'] = dataset['Flag_Mixed_Breed'].fillna(0)
    
    # age < 3 months
    dataset['Flag_Age_le_3'] = np.nan
    dataset['Flag_Age_le_3'] = dataset['Flag_Age_le_3'].mask(dataset['Age'] <= 3, 1)
    dataset['Flag_Age_le_3'] = dataset['Flag_Age_le_3'].fillna(0)
    
    # Name exists
    dataset['Flag_Name'] = np.nan
    dataset['Flag_Name'] = dataset['Flag_Name'].mask(dataset['Name'].isnull(), 1)
    dataset['Flag_Name'] = dataset['Flag_Name'].fillna(0)
    return dataset

def standardiseVariables(data):
#    std_cols = ['Age', 'Quantity', 'Fee', 'VideoAmt', 'PhotoAmt', 'RescuerID_Count']
    std_cols = ['Age', 'Quantity', 'Fee', 'VideoAmt', 'PhotoAmt']
    dataset = data.copy()
    dataset[[c + "_std" for c in std_cols]] = StandardScaler().fit_transform(dataset[std_cols])
    return dataset

def oneHotEncode(data):
    dummy_cols = ['Type','Gender','Health','FurLength','State','Color1','Color2','Color3']
    dataset = data.copy()
    return pd.get_dummies(dataset, columns=dummy_cols)

def getTargetEncodingMap(data, col, response):
    target_map = data.groupby([col])[response].mean().to_dict()
    return target_map

def targetEncodeFeature(data, col, mapping):
    # yes i know this is the same as above but im literally so lazy so ctrl c + v
    dataset = data.copy()
    dataset[col + '_target_encode'] = dataset[col].map(mapping).fillna(0)
    return dataset

def targetEncodeFeatures(data, cols, response):
    # yes i know this is the same as above but im literally so lazy so ctrl c + v
    dataset = data.copy()
    mappings = dict()
    for col in cols:
        mapping = getTargetEncodingMap(dataset, col, response)
        mappings[col] = mapping
        dataset = targetEncodeFeature(dataset, col, mapping)
    return dataset, mappings

def dropNonFeatureColumns(data):
    non_feature_cols = [
        'PetID',
        'Name',
        'RescuerID',
        'Age',
        'Quantity',
        'Fee',
        'PhotoAmt',
        'VideoAmt',
        'Description',
        'Breed1',
        'Breed2'
    ]
    dataset = data.copy()
    return dataset.drop(columns=non_feature_cols)

def preProcessTrain(data):
    mappings = dict()
    data, mappings['RescuerID'] = createRescuerIDCount(data)
    data = createDummyVariables(data)
    data = standardiseVariables(data)
    data = oneHotEncode(data)
    data, target_mappings = targetEncodeFeatures(data,
                                                cols=['Breed1', 'Breed2'],
                                                response='AdoptionSpeed')
    mappings.update(target_mappings)
    data = dropNonFeatureColumns(data)
    return data, mappings

def preProcessTest(data, mappings, train_cols):
    train_cols = list(train_cols)
    train_cols.remove('AdoptionSpeed')
    data = data.fillna(0)
    data = encodeRescuerIDCount(data, mappings['RescuerID'])
    data = createDummyVariables(data)
    data = standardiseVariables(data)
    data = oneHotEncode(data)
    data = targetEncodeFeature(data, 'Breed1', mappings['Breed1'])
    data = targetEncodeFeature(data, 'Breed2', mappings['Breed2'])
    data = dropNonFeatureColumns(data)
    data = data[train_cols]
    return data

In [ ]:
train, mappings = preProcessTrain(train_tabular_df)
test = preProcessTest(test_tabular_df, mappings, train.columns)

In [ ]:
train.shape

## 3. Train and Evaluate

1. Split datasets into features and responses
2. Fit on training data
3. Test on unseen test data
4. Create submission file and submit.

In [ ]:
def splitDataXY(dataset, response_column):
    temp = dataset.copy()
    Y_train = temp[response_column]
    X_train = temp.drop(columns=[response_column])
    return X_train, Y_train

We use the values obtained from Gridsearch (code not run here)

```
grid = {'C': [0.0001, 0.001, 0.01, 1, 10, 100],
        'class_weight': [None, 'balanced']}

logreg = lm.LogisticRegression(solver='saga', max_iter=10000)
gridsearch_model = GridSearchCV(logreg,
                                 grid, 
#                                 cv=5, #
                                 scoring=quadratic_kappa_scorer, 
                                 return_train_score=True,
                                 verbose=3,
                                 n_jobs=-1)
gridsearch_model.fit(X_train, Y_train)
```

```
$ Average time taken per fit: 61.51685015360514
$ Best Parameters: {'C': 10, 'class_weight': None}
$ Best Quadratic Kappa 0.3104591557225801
```

In [ ]:
X_train_df, Y_train_df = splitDataXY(train, 'AdoptionSpeed')
X_train = X_train_df.to_numpy()
Y_train = Y_train_df.to_numpy()

logreg = lm.LogisticRegression(solver='saga', 
                               penalty= 'l1', 
                               C=10, 
                               class_weight=None,
                               max_iter=10000, 
                               n_jobs=-1)

logreg.fit(X_train, Y_train)

predicted = logreg.predict(test)

print("Training score:", cohen_kappa_score(logreg.predict(X_train), Y_train, weights='quadratic'))

In [ ]:
for i, c in enumerate(X_train_df.columns):
    print(f'{i}. {c}')

In [ ]:
print("Training score:", cohen_kappa_score(logreg.predict(X_train), Y_train, weights='quadratic'))

In [ ]:
submission = pd.DataFrame.from_dict({'PetID': test_tabular_df['PetID'],
                                     'AdoptionSpeed': predicted})
print(submission.shape)
submission.head()

In [ ]:
submission.to_csv("/kaggle/working/submission.csv", index=False)

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))